<a href="https://colab.research.google.com/github/xy2119/Causal_Knowledge_GNN/blob/main/notebooks/3_causal_weighting_embedding(10d)_ate(13d).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This notebook uses GCN and GAT to Estimate Average Treatment Effect based on BN Connectivity and Causally Weighted Feature Embeddings

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from gensim.models import word2vec
import pandas as pd
import numpy as np 
import random
random.seed(2022)
np.random.seed(2022)

In [ ]:
!pip install --upgrade gensim
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
df_train=pd.read_csv("criteo_sampled/criteo_train.csv",index_col=0)
df_test=pd.read_csv("criteo_sampled/criteo_test.csv",index_col=0)

In [ ]:
# load causal weighting
ate_list=[]
ate=pd.read_excel("feats_ate_x13.xlsx",index_col=0)
for i in [c for c in df_train.columns[:-4]]+["visit","treatment"]:
    ate_list.append(float(ate[ate['Feature']==i]["ATE"].values))
ate_list

[-0.01388679662724193,
 0.05071538371534517,
 0.001328853128405821,
 -0.009331114629769072,
 0.1933108932875426,
 0.0008912893595676433,
 -0.0002958249515911775,
 -0.009201954095841196,
 0.1034574925533489,
 0.0741312293943781,
 -0.01664079180275199,
 0.1331562899243786,
 0.9310119697738223,
 0.007447354046319239]

In [ ]:
# load edge index
edge_index=pd.read_csv('edge_index_criteo.csv')
edge_index=torch.from_numpy(np.transpose(np.array(edge_index)))

In [ ]:
# load node embedding
model_dw=word2vec.Word2Vec.load("deepwalk_10d_x13.model")
model_n2v=word2vec.Word2Vec.load("Node2Vec_10d_x13.model")

lst_dw=[]
lst_n2v=[]
for i in range(14):
    lst_dw.append(model_dw.wv[i])
    lst_n2v.append(model_n2v.wv[i])
len(lst_n2v)

14

In [ ]:
from scipy.stats import entropy
def R2(y_predicted, y_actual):
    y_predicted = np.asarray(y_predicted, dtype=float)
    y_actual = np.asarray(y_actual, dtype=float)

    R2 = 1 - np.sum(np.square(y_actual-y_predicted)) / np.sum(np.square(y_actual-np.mean(y_actual)))
    return R2

def MAE(y_predicted, y_actual):
    y_predicted = np.asarray(y_predicted, dtype=float)
    y_actual = np.asarray(y_actual, dtype=float)

    MAE = np.mean(abs(y_actual-y_predicted))
    return MAE

def RMSE(y_predicted, y_actual):
    y_predicted = np.asarray(y_predicted, dtype=float)
    y_actual = np.asarray(y_actual, dtype=float)

    RMSE = np.sqrt(np.mean(np.square(y_actual-y_predicted)))
    return RMSE

def CVRMSE(y_predicted, y_actual):
    y_predicted = np.asarray(y_predicted, dtype=float)
    y_actual = np.asarray(y_actual, dtype=float)

    RMSE = np.sqrt(np.mean(np.square(y_actual-y_predicted)))
    CVRMSE = RMSE/np.mean(y_actual)
    return CVRMSE

def MAPE(y_predicted, y_actual):
    y_predicted = np.asarray(y_predicted, dtype=float)
    y_actual = np.asarray(y_actual, dtype=float)

    MAPE=np.mean(abs((y_actual-y_predicted)/y_actual))
    return MAPE

def MSE(y_predicted, y_actual):
    y_predicted = np.asarray(y_predicted, dtype=float)
    y_actual = np.asarray(y_actual, dtype=float)

    MSE = np.mean(abs(y_actual-y_predicted)**2)
    return MSE

def kl_divergence(p, q):

    p = np.asarray(p, dtype=float)
    q = np.asarray(q, dtype=float)
    return np.sum(np.where(p != 0, p * np.log(p / q), 0))

def kl_divergence(y_predicted, y_actual):

    stacked_values = np.hstack((y_predicted, y_actual))
    stacked_low = np.percentile(stacked_values, 0.1)
    stacked_high = np.percentile(stacked_values, 99.9)
    bins = np.linspace(stacked_low, stacked_high, 100)

    distr = np.histogram(y_predicted, bins=bins)[0]
    distr = np.clip(distr / distr.sum(), 0.001, 0.999)
    true_distr = np.histogram( y_actual, bins=bins)[0]
    true_distr = np.clip(true_distr / true_distr.sum(), 0.001, 0.999)

    kl = entropy(distr, true_distr)
    return kl 

In [ ]:
import torch
from torch_geometric.data import InMemoryDataset, download_url


class Mytrain(InMemoryDataset):
    def __init__(self, root,transform=None, pre_transform=None):
        super().__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])
       # ,self.edge_index,self.y=x,edge_index,y
        
    
    @property
    def raw_file_names(self):
        return ['some_file_1', 'some_file_2', ...]
    
    @property
    def processed_file_names(self):
        return ['data.pt']

    def process(self):
        data_list=[]
        weighted_feats=[]
        for i in range(x_train.shape[0]):
            Edge_index = edge_index.type(torch.long)
            X =x_train[i]
            if feats_mode =='causal':
                t=torch.zeros(14,25) # 25 dimensions = 1+10+14 = node number, node embedding, features
              
                causal_weighted= np.multiply(np.array(X),np.array(ate_list))
                weighted_feats.append(causal_weighted)
                
                for j in range(14): # for j-th feature
                  t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # node (1d)
                                                        list(lst_dw[int(j)]), # node embeddings (10d), prior embeddings from DeepWalk/Node2Vec
                                                        list(causal_weighted) # feature (10d), causally weighted features        
                                                        ))) 
            elif feats_mode =='equal':
                t=torch.zeros(14,25) # 25 dimensions = 1+10+14 = node number, node embedding, features
              
                weighted= np.array(X)
                weighted_feats.append(weighted)
                
                for j in range(14): # for j-th feature
                  t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # node (1d)
                                                        list(lst_dw[int(j)]), # node embeddings (10d), prior embeddings from DeepWalk/Node2Vec
                                                        list(weighted) # feature (10d), equally weighted features        
                                                        ))) 
            else:
                t=torch.zeros(14,11) # 11 dimensions = 1+10 = node number, node embedding
                                    
                
                for j in range(14):
                  t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # node (1d)
                                                        list(lst_dw[int(j)]), # node embeddings (10d), prior embeddings from DeepWalk/Node2Vec
                                                        ))) 
                  

            Y = y_train[i].reshape(-1,1).to(torch.float32)
            data = Data(x=t, edge_index=Edge_index, y=Y)
      
            data_list.append(data)

        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])


class Mytest(InMemoryDataset):
    def __init__(self, root,transform=None, pre_transform=None):
        super().__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])
       # ,self.edge_index,self.y=x,edge_index,y
        
    
    @property
    def raw_file_names(self):
        return ['some_file_1', 'some_file_2', ...]
    
    @property
    def processed_file_names(self):
        return ['data.pt']

    def process(self):
        data_list=[]
        weighted_feats=[]
        for i in range(x_test.shape[0]):
            Edge_index = edge_index.type(torch.long)
            X =x_test[i]

            if feats_mode =='causal':
                t=torch.zeros(14,25) # 25 dimensions = 1+10+14 = node number, node embedding, features
              
                causal_weighted= np.multiply(np.array(X),np.array(ate_list))
                weighted_feats.append(causal_weighted)
                
                for j in range(14): # for j-th feature
                  t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # node (1d)
                                                        list(lst_dw[int(j)]), # node embeddings (10d), prior embeddings from DeepWalk/Node2Vec
                                                        list(causal_weighted) # feature (10d), causally weighted features        
                                                        ))) 
            elif feats_mode =='equal':
                t=torch.zeros(14,25) # 25 dimensions = 1+10+14 = node number, node embedding, features
              
                weighted= np.array(X)
                weighted_feats.append(weighted)
                
                for j in range(14): # for j-th feature
                  t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # node (1d)
                                                        list(lst_dw[int(j)]), # node embeddings (10d), prior embeddings from DeepWalk/Node2Vec
                                                        list(weighted) # feature (10d), equally weighted features        
                                                        ))) 
            else:
                t=torch.zeros(14,11) # 11 dimensions = 1+10 = node number, node embedding
                                    
                
                for j in range(14):
                  t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # node (1d)
                                                        list(lst_dw[int(j)]), # node embeddings (10d), prior embeddings from DeepWalk/Node2Vec
                                                        ))) 
            
            Y = y_test[i].reshape(-1,1).to(torch.float32)
            data = Data(x=t, edge_index=Edge_index, y=Y)
        
            data_list.append(data)

        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [ ]:
feats_mode='causal'
#feats_mode ='equal' 
#feats_mode ='noweighting' 

col=df_train.columns
y_train=torch.from_numpy(np.array(df_train['y'])).reshape(df_train.shape[0],1).to(torch.float32)
x_train=torch.from_numpy(np.array(df_train[[i for i in col[:-4]]+["visit","T"]])).to(torch.float32)
Mydata_train=Mytrain(".\mydata_sampled_ate\MYdata_train")

y_test=torch.from_numpy(np.array(df_test['y'])).reshape(df_test.shape[0],1).to(torch.float32)
x_test=torch.from_numpy(np.array(df_test[[i for i in col[:-4]]+["visit","T"]])).to(torch.float32)
Mydata_test=Mytest(".\mydata_sampled_ate\MYdata_test")

# GCN

In [ ]:
def train():
    model.train()
    loss_all = 0
    y_actual = []
    y_predicted = []
    loss_all=0
    for data in train_loader:
        loss = 0
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        label = data.y.to(device)
        loss = crit(output, label)
        loss.backward()
        loss_all += data.num_graphs * loss.item()
        optimizer.step()
        y_actual +=(label).cpu().detach().ravel().tolist()
        y_predicted +=(output).cpu().detach().ravel().tolist()
    
    
    loss=loss_all/len(Mydata_train)
    r2=R2(y_predicted, y_actual)
    mse = MSE(y_predicted, y_actual)
    kl=kl_divergence(y_predicted, y_actual)

    print("R2:%f" % (R2(y_predicted, y_actual)),end='  ')
    print("MSE:%f" % (MSE(y_predicted, y_actual)),end='  ')
    print("KL:%f" % (kl_divergence(y_predicted, y_actual)),end='  ')
    print("MAE:%f" % (MAE(y_predicted, y_actual)),end='  ')
    print("RMSE:%f" % (RMSE(y_predicted, y_actual)),end='  ')
    print("CVRMSE:%f" % (CVRMSE(y_predicted, y_actual)),end='  ')

    return loss,r2,mse,kl


def val():
    model.eval()
    y_actual = []
    y_predicted = []
    loss_all=0
    for data in test_loader:
      loss = 0
      data = data.to(device)
      output = model(data)
      label = data.y.to(device)
      y_actual +=(label).cpu().detach().ravel().tolist()
      y_predicted +=(output).cpu().detach().ravel().tolist()
      loss = crit(output, label)
      loss_all += loss.item()

    
    loss = loss_all / len(Mydata_test)
    r2=R2(y_predicted, y_actual)
    mse = MSE(y_predicted, y_actual)
    kl=kl_divergence(y_predicted, y_actual)

    print("R2:%f" % (R2(y_predicted, y_actual)),end='  ')
    print("MSE:%f" % (MSE(y_predicted, y_actual)),end='  ')
    print("KL:%f" % (kl_divergence(y_predicted, y_actual)),end='  ')
    print("MAE:%f" % (MAE(y_predicted, y_actual)),end='  ')
    print("RMSE:%f" % (RMSE(y_predicted, y_actual)),end='  ')
    print("CVRMSE:%f" % (CVRMSE(y_predicted, y_actual)),end='  ')

    return loss,r2, mse,kl

In [ ]:
import torch
from torch_scatter import scatter_add
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(25, 64)
        self.conv2 = GCNConv(64, 10)
        # self-attention layer
        
        self.f1 = torch.nn.Linear(140,32)
        self.f2 = torch.nn.Linear(32,1)
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = self.attention
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = x.reshape(-1,140)
        x = self.f1(x)
        x = self.f2(x)
        return x

In [ ]:
num_epochs = 2560
batch_size = 512
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.002, weight_decay=5e-4)
crit = F.mse_loss
train_loader = DataLoader(Mydata_train, batch_size=batch_size)
test_loader = DataLoader(Mydata_test, batch_size=512)

for epoch in range(num_epochs):
    loss,r2,mse,kl=train()
    if epoch %5==0:      
        print('train_loss:')
        print(loss)

        loss,r2,mse,kl=val()
        print('test_loss:')
        print(loss)

y_predicted = []
for data in test_loader:
    loss = 0
    data = data.to(device)
    output = model(data)
    label = data.y.to(device)
    y_predicted +=(output).cpu().detach().ravel().tolist()
df_test["y_hat"]=y_predicted

In [ ]:
train_loader = DataLoader(Mydata_train, batch_size=batch_size)
for epoch in range(num_epochs):
    loss,r2,mse,kl=train()
    if epoch %5==0:      
        print('train_loss:')
        print(loss)

        loss,r2,mse,kl=val()
        print('test_loss:')
        print(loss)

In [ ]:
# estimate area under the uplift curve (AUUC)
from causalml.metrics import *
uplift=df_test.copy()
tau_hat=pd.concat([t0,t1], axis=0, join="inner")
uplift=pd.concat([uplift,tau_hat], axis=1, join="inner")
uplift = uplift.loc[:,~uplift.columns.duplicated()]

auuc=auuc_score(uplift, outcome_col='y', treatment_col='T')
gcn_auuc=pd.DataFrame(auuc[["y_hat","Random"]],columns=['auuc'])
gcn_auuc

,auuc
y_hat,0.732616
Random,0.499185


# GAT

In [ ]:
from uuid import RFC_4122
import torch
import math
from torch_geometric.nn import MessagePassing
from torch_geometric.nn import GATConv

from torch_geometric.utils import add_self_loops,degree
from torch_geometric.datasets import Planetoid
import ssl
import torch.nn.functional as F


class Net(torch.nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.gat1=GATConv(in_channels=25,out_channels=8,heads=8,dropout=0.6)
        self.gat2=GATConv(in_channels=64,out_channels=10,heads=1,dropout=0.6)
        self.f1 = torch.nn.Linear(140,32)
        self.f2 = torch.nn.Linear(32,1)
    def forward(self,data):
        x,edge_index=data.x, data.edge_index
        x=self.gat1(x,edge_index)
        x=self.gat2(x,edge_index)
        x=x.reshape(-1,140)
        x = self.f1(x)
        x = self.f2(x)
        return x

ssl._create_default_https_context = ssl._create_unverified_context
def train():
    model.train()
    loss_all = 0
    y_actual = []
    y_predicted = []
    loss_all=0
    for data in train_loader:
        loss = 0
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        label = data.y.to(device)
        loss = crit(output, label)
        loss.backward()
        loss_all += data.num_graphs * loss.item()
        optimizer.step()
        y_actual +=(label).cpu().detach().ravel().tolist()
        y_predicted +=(output).cpu().detach().ravel().tolist()
    
    
    loss=loss_all/len(Mydata_train)
    r2=R2(y_predicted, y_actual)
    mse = MSE(y_predicted, y_actual)
    kl=kl_divergence(y_predicted, y_actual)

    print("R2:%f" % (R2(y_predicted, y_actual)),end='  ')
    print("MSE:%f" % (MSE(y_predicted, y_actual)),end='  ')
    print("KL:%f" % (kl_divergence(y_predicted, y_actual)),end='  ')
    print("MAE:%f" % (MAE(y_predicted, y_actual)),end='  ')
    print("RMSE:%f" % (RMSE(y_predicted, y_actual)),end='  ')
    print("CVRMSE:%f" % (CVRMSE(y_predicted, y_actual)),end='  ')

    return loss,r2,mse,kl


def val():
    model.eval()
    y_actual = []
    y_predicted = []
    loss_all=0
    for data in test_loader:
      loss = 0
      data = data.to(device)
      output = model(data)
      label = data.y.to(device)
      y_actual +=(label).cpu().detach().ravel().tolist()
      y_predicted +=(output).cpu().detach().ravel().tolist()
      loss = crit(output, label)
      loss_all += loss.item()

    
    loss = loss_all / len(Mydata_test)
    r2=R2(y_predicted, y_actual)
    mse = MSE(y_predicted, y_actual)
    kl=kl_divergence(y_predicted, y_actual)

    print("R2:%f" % (R2(y_predicted, y_actual)),end='  ')
    print("MSE:%f" % (MSE(y_predicted, y_actual)),end='  ')
    print("KL:%f" % (kl_divergence(y_predicted, y_actual)),end='  ')
    print("MAE:%f" % (MAE(y_predicted, y_actual)),end='  ')
    print("RMSE:%f" % (RMSE(y_predicted, y_actual)),end='  ')
    print("CVRMSE:%f" % (CVRMSE(y_predicted, y_actual)),end='  ')

    return loss,r2, mse,kl



num_epochs = 256
batch_size = 512
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.002, weight_decay=5e-4)
crit = F.mse_loss
train_loader = DataLoader(Mydata_train, batch_size=batch_size)
test_loader = DataLoader(Mydata_test, batch_size=512)

for epoch in range(num_epochs):
    loss,r2,mse,kl=train()
    if epoch %5==0:      
        print('train_loss:')
        print(loss)

        loss,r2,mse,kl=val()
        print('test_loss:')
        print(loss)

y_predicted = []
for data in test_loader:
    loss = 0
    data = data.to(device)
    output = model(data)
    label = data.y.to(device)
    y_predicted +=(output).cpu().detach().ravel().tolist()
df_test["y_hat"]=y_predicted

R2:-0.813384  MSE:0.005295  KL:2.094833  MAE:0.017525  RMSE:0.072765  CVRMSE:24.848187  train_loss:
0.005294719280111404
R2:0.072054  MSE:0.002798  KL:0.126590  MAE:0.005984  RMSE:0.052893  CVRMSE:17.490782  test_loss:
5.497692996261892e-06
R2:0.060488  MSE:0.002743  KL:2.104842  MAE:0.009919  RMSE:0.052375  CVRMSE:17.885496  R2:0.057913  MSE:0.002751  KL:1.238602  MAE:0.008663  RMSE:0.052447  CVRMSE:17.909985  R2:0.060253  MSE:0.002744  KL:1.248182  MAE:0.008433  RMSE:0.052382  CVRMSE:17.887734  R2:0.064215  MSE:0.002732  KL:1.533469  MAE:0.008456  RMSE:0.052271  CVRMSE:17.849982  R2:0.064579  MSE:0.002731  KL:1.586445  MAE:0.008461  RMSE:0.052261  CVRMSE:17.846508  train_loss:
0.0027312411740449236
R2:0.056647  MSE:0.002844  KL:0.100752  MAE:0.005517  RMSE:0.053330  CVRMSE:17.635387  test_loss:
5.589489941400884e-06
R2:0.064812  MSE:0.002731  KL:1.744580  MAE:0.008459  RMSE:0.052255  CVRMSE:17.844289  R2:0.065108  MSE:0.002730  KL:1.738378  MAE:0.008456  RMSE:0.052247  CVRMSE:17.8414

R2:0.064456  MSE:0.002732  KL:1.592869  MAE:0.008475  RMSE:0.052265  CVRMSE:17.847686  R2:0.064599  MSE:0.002731  KL:1.694082  MAE:0.008464  RMSE:0.052261  CVRMSE:17.846322  R2:0.064635  MSE:0.002731  KL:1.742830  MAE:0.008471  RMSE:0.052260  CVRMSE:17.845979  R2:0.064600  MSE:0.002731  KL:1.617236  MAE:0.008486  RMSE:0.052261  CVRMSE:17.846311  R2:0.064951  MSE:0.002730  KL:1.751280  MAE:0.008426  RMSE:0.052251  CVRMSE:17.842962  train_loss:
0.002730155742312368
R2:0.056884  MSE:0.002843  KL:0.100740  MAE:0.005550  RMSE:0.053324  CVRMSE:17.633171  test_loss:
5.588077468592933e-06
R2:0.064745  MSE:0.002731  KL:1.648253  MAE:0.008481  RMSE:0.052257  CVRMSE:17.844927  R2:0.064765  MSE:0.002731  KL:1.741393  MAE:0.008428  RMSE:0.052256  CVRMSE:17.844735  R2:0.065008  MSE:0.002730  KL:1.755607  MAE:0.008450  RMSE:0.052249  CVRMSE:17.842421  R2:0.064726  MSE:0.002731  KL:1.748324  MAE:0.008494  RMSE:0.052257  CVRMSE:17.845111  R2:0.064690  MSE:0.002731  KL:1.677063  MAE:0.008428  RMSE:0.052

R2:0.064303  MSE:0.002732  KL:1.664241  MAE:0.008439  RMSE:0.052269  CVRMSE:17.849148  R2:0.064583  MSE:0.002731  KL:1.576043  MAE:0.008445  RMSE:0.052261  CVRMSE:17.846474  R2:0.064168  MSE:0.002732  KL:1.796350  MAE:0.008418  RMSE:0.052273  CVRMSE:17.850430  R2:0.064346  MSE:0.002732  KL:1.714394  MAE:0.008441  RMSE:0.052268  CVRMSE:17.848737  R2:0.065047  MSE:0.002730  KL:1.782470  MAE:0.008482  RMSE:0.052248  CVRMSE:17.842051  train_loss:
0.002729876980947571
R2:0.058913  MSE:0.002837  KL:0.106155  MAE:0.006180  RMSE:0.053266  CVRMSE:17.614195  test_loss:
5.5759237337438345e-06
R2:0.064603  MSE:0.002731  KL:1.704743  MAE:0.008488  RMSE:0.052261  CVRMSE:17.846283  R2:0.065151  MSE:0.002730  KL:1.568244  MAE:0.008501  RMSE:0.052245  CVRMSE:17.841057  R2:0.064490  MSE:0.002732  KL:1.684507  MAE:0.008433  RMSE:0.052264  CVRMSE:17.847359  R2:0.064327  MSE:0.002732  KL:1.678769  MAE:0.008466  RMSE:0.052268  CVRMSE:17.848919  R2:0.064538  MSE:0.002731  KL:1.686070  MAE:0.008441  RMSE:0.05

R2:0.064688  MSE:0.002731  KL:1.731077  MAE:0.008439  RMSE:0.052258  CVRMSE:17.845475  R2:0.064379  MSE:0.002732  KL:1.767459  MAE:0.008429  RMSE:0.052267  CVRMSE:17.848424  R2:0.064461  MSE:0.002732  KL:1.777455  MAE:0.008439  RMSE:0.052265  CVRMSE:17.847634  R2:0.064899  MSE:0.002730  KL:1.683874  MAE:0.008465  RMSE:0.052252  CVRMSE:17.843460  R2:0.063806  MSE:0.002733  KL:1.786041  MAE:0.008441  RMSE:0.052283  CVRMSE:17.853883  train_loss:
0.002733499015558268
R2:0.056011  MSE:0.002846  KL:0.099016  MAE:0.005763  RMSE:0.053348  CVRMSE:17.641328  test_loss:
5.593251218773543e-06
R2:0.064669  MSE:0.002731  KL:1.590624  MAE:0.008481  RMSE:0.052259  CVRMSE:17.845651  R2:0.064686  MSE:0.002731  KL:1.735661  MAE:0.008474  RMSE:0.052258  CVRMSE:17.845490  R2:0.065101  MSE:0.002730  KL:1.621649  MAE:0.008488  RMSE:0.052247  CVRMSE:17.841530  R2:0.064795  MSE:0.002731  KL:1.760341  MAE:0.008435  RMSE:0.052255  CVRMSE:17.844450  R2:0.064473  MSE:0.002732  KL:1.624434  MAE:0.008470  RMSE:0.052

In [ ]:
# estimate area under the uplift curve (AUUC)
from causalml.metrics import *
uplift=df_test.copy()
tau_hat=pd.concat([t0,t1], axis=0, join="inner")
uplift=pd.concat([uplift,tau_hat], axis=1, join="inner")
uplift = uplift.loc[:,~uplift.columns.duplicated()]

auuc=auuc_score(uplift, outcome_col='y', treatment_col='T', treatment_effect_col='tau')
gat_auuc=pd.DataFrame(auuc[["y_hat","Random"]],columns=['auuc'])
gat_auuc

,auuc
y_hat,0.880700
Random,0.499185


In [ ]:
result=pd.DataFrame(columns=[['AUUC']])
result.loc['S Learner(LR)','AUUC']=0.497983
result.loc['S Learner(XGB)','AUUC']=0.875572
result.loc['S Learner(LGBM)','AUUC']=0.883033

result.loc['GCN (Struct)','AUUC']=0.501865
result.loc['GCN (Struct+Feature)','AUUC']=0.721959
result.loc['GCN (Struct+Causal Weighting)','AUUC']=gcn_auuc.loc["y_hat"].values

result.loc['GAT (Struct)','AUUC']=0.544286
result.loc['GAT (Struct+Feature)','AUUC']=0.847630
result.loc['GAT (Struct+Causal Weighting)','AUUC']=gat_auuc.loc["y_hat"].values

result

,AUUC
S Learner(LR),0.497983
S Learner(XGB),0.875572
S Learner(LGBM),0.883033
GCN (Struct),0.501865
GCN (Struct+Feature),0.721959
GCN (Struct+Causal Weighting),0.732616
GAT (Struct),0.544286
GAT (Struct+Feature),0.84763
GAT (Struct+Causal Weighting),0.8807


In [ ]:
result=pd.DataFrame(columns=[['GCN','GAT']])
result.loc['equal weighting','GCN']=0.721959
result.loc['equal weighting','GAT']=0.847630

result.loc['causal weighting','GCN']=gcn_auuc.loc["y_hat"].values
result.loc['causal weighting','GAT']=gat_auuc.loc["y_hat"].values
result

,GCN,GAT
equal weighting,0.721959,0.84763
causal weighting,0.732616,0.8807
